-sandbox
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://camo.githubusercontent.com/290ee18049382f170e5f63956a389da081a8c16f/68747470733a2f2f73332e616d617a6f6e6177732e636f6d2f636f6d2e726176656e7061636b2e636d732f70616765732f6a702d6d6f7267616e2d6269672d646174612d61692d35343637413139302e6a7067"  style="width: 600px; height: 163px">
</div>

%fs unmount /mnt/deepfin3

%fs ls /mnt/

In [5]:
import requests
import os
from bs4 import BeautifulSoup
from azure.storage.blob import BlockBlobService

In [6]:
wasb_account_key = 'TODO'
wasb_account_name = 'TODO'

# Create the BlockBlockService that is used to call the Blob service for the storage account
block_blob_service = BlockBlobService(account_name=wasb_account_name, account_key=wasb_account_key)

# Create a container called 'quickstartblobs'.
container_name ='deepfin3'
if block_blob_service.exists(container_name) is False:
  block_blob_service.create_container(container_name)      

In [7]:
def scrape_find_zips(url):
    """ scrape the given url for any zip files in href tags """
    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.1) Gecko/2008071615 Fedora/3.0.1-1.fc9 Firefox/3.0.1',
    }

    response = requests.get(url, headers=headers)
    #print(response.status_code)
    #print(response.text)
    soup = BeautifulSoup(response.text, 'html.parser')
    all_tags = soup.find_all('a', href=True)
    tag_dict = {}
    for tag in all_tags:
      if '.zip' in tag['href']:
          tag_dict[tag.get_text()] = tag['href']
    return tag_dict


In [8]:
url = "https://www.asxhistoricaldata.com/archive/"

tag_dict = scrape_find_zips(url)

print(tag_dict.values())

dict_values(['https://www.asxhistoricaldata.com/data/2007-2012.zip', 'https://www.asxhistoricaldata.com/data/2017july-december.zip', 'https://www.asxhistoricaldata.com/data/2017jan-june.zip', 'https://www.asxhistoricaldata.com/data/2013-2016.zip', 'https://www.asxhistoricaldata.com/data/1997-2006.zip'])

In [9]:
from azure.storage.file import ContentSettings

def scrape_downloader(urls, dest):
    """ download all files listed in urls {} dict 
    Input:
        url = base url
        urls= target paths to append to base url
        dest= local path to store files
    """
    for k_name, v_path in urls.items():
        src = v_path        
        
        headers = {
            'User-Agent': 'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.1) Gecko/2008071615 Fedora/3.0.1-1.fc9 Firefox/3.0.1',
        }
        r = requests.get(src, headers=headers)  
        if r is not None:
          block_blob_service.create_blob_from_bytes(dest, k_name + '.zip', r.content)            
          print(src)

In [10]:
scrape_downloader(tag_dict, container_name)

https://www.asxhistoricaldata.com/data/2007-2012.zip
https://www.asxhistoricaldata.com/data/2017july-december.zip
https://www.asxhistoricaldata.com/data/2017jan-june.zip
https://www.asxhistoricaldata.com/data/2013-2016.zip
https://www.asxhistoricaldata.com/data/1997-2006.zip

In [11]:
blobs = block_blob_service.list_blobs(container_name)

for blob in blobs:
  print(blob.name)

1997 - 2006.zip
2007 - 2012.zip
2013 - 2016.zip
Jan - June.zip
July - December.zip

In [12]:
import zipfile
import csv
import datetime
from io import BytesIO, TextIOWrapper
from pyspark.sql import Row


#get all files in container
f_zip = [ blob.name for blob in blobs]

#Create empty lists
newlines = lines = header = all_lines = []

#limit to single 
for f in f_zip: #[:1]: limit
    #only go through zip files in container 
    if f.endswith(".zip"):
      f_bytes = block_blob_service.get_blob_to_bytes(container_name, f)
      with zipfile.ZipFile(BytesIO(f_bytes.content)) as main_zip:  
        for inner_file in main_zip.namelist():   
            #print(inner_file)
            #read inner zip file
            zfiledata = BytesIO(main_zip.read(inner_file))
            inner_csv = TextIOWrapper(zfiledata, encoding='utf8', newline='')            
            reader = csv.reader(inner_csv, delimiter=',', dialect=csv.unix_dialect)
            
            for row, line in enumerate(reader):
                #print(line)
                # 'Ticker', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume'
                lines.append(Row(Ticker=line[0], Date=datetime.datetime.strptime(line[1], '%Y%m%d') , High=float(line[2]), Low=float(line[3]), Close=float(line[4]), Volume=float(line[5])))
            

In [13]:
df = sqlContext.createDataFrame(sc.parallelize(lines))

In [14]:
df.printSchema()

root
-- Close: double (nullable = true)
-- Date: timestamp (nullable = true)
-- High: double (nullable = true)
-- Low: double (nullable = true)
-- Ticker: string (nullable = true)
-- Volume: double (nullable = true)

In [15]:
df.count()

Out[ 8 ]: 6475470

In [16]:
%sql drop table if exists asxhistoric;

In [17]:
df.write.saveAsTable("asxhistoric")

In [18]:
from pyspark.sql.functions import year

focus_df = ((df 
  .select("Date", "Close", "Ticker") 
  .filter("Ticker = 'AAA'")
  .filter(year("Date") == '1998')
))

In [19]:
focus_df.count()

Out[ 11 ]: 254

In [20]:
display(focus_df)

Date Close Ticker 1998-01-02T00:00:00.000+0000 1.38 AAA 1998-01-05T00:00:00.000+0000 1.34 AAA 1998-01-06T00:00:00.000+0000 1.27 AAA 1998-01-07T00:00:00.000+0000 1.36 AAA 1998-01-08T00:00:00.000+0000 1.375 AAA 1998-01-09T00:00:00.000+0000 1.42 AAA 1998-01-12T00:00:00.000+0000 1.48 AAA 1998-01-13T00:00:00.000+0000 1.5 AAA 1998-01-14T00:00:00.000+0000 1.568 AAA 1998-01-15T00:00:00.000+0000 1.49 AAA 1998-01-16T00:00:00.000+0000 1.54 AAA 1998-01-19T00:00:00.000+0000 1.58 AAA 1998-01-20T00:00:00.000+0000 1.59 AAA 1998-01-21T00:00:00.000+0000 1.63 AAA 1998-01-22T00:00:00.000+0000 1.71 AAA 1998-01-23T00:00:00.000+0000 1.68 AAA 1998-01-27T00:00:00.000+0000 1.77 AAA 1998-01-28T00:00:00.000+0000 1.82 AAA 1998-01-29T00:00:00.000+0000 1.96 AAA 1998-01-30T00:00:00.000+0000 1.94 AAA 1998-02-02T00:00:00.000+0000 1.88 AAA 1998-02-03T00:00:00.000+0000 1.83 AAA 1998-02-04T00:00:00.000+0000 1.76 AAA 1998-02-05T00:00:00.000+0000 1.84 AAA 1998-02-06T00:00:00.000+0000 1.86 AAA 1998-02-09T00:00:00.000+0000 1.88 AAA 1998-02-10T00:00:00.000+0000 1.969 AAA 1998-02-11T00:00:00.000+0000 1.98 AAA 1998-02-12T00:00:00.000+0000 1.97 AAA 1998-02-13T00:00:00.000+0000 1.93 AAA 1998-02-16T00:00:00.000+0000 1.87 AAA 1998-02-17T00:00:00.000+0000 1.84 AAA 1998-02-18T00:00:00.000+0000 1.84 AAA 1998-02-19T00:00:00.000+0000 1.86 AAA 1998-02-20T00:00:00.000+0000 1.95 AAA 1998-02-23T00:00:00.000+0000 1.96 AAA 1998-02-24T00:00:00.000+0000 1.92 AAA 1998-02-25T00:00:00.000+0000 1.9 AAA 1998-02-26T00:00:00.000+0000 1.93 AAA 1998-02-27T00:00:00.000+0000 1.92 AAA 1998-03-02T00:00:00.000+0000 1.99 AAA 1998-03-03T00:00:00.000+0000 1.98 AAA 1998-03-04T00:00:00.000+0000 1.96 AAA 1998-03-05T00:00:00.000+0000 1.96 AAA 1998-03-06T00:00:00.000+0000 1.91 AAA 1998-03-09T00:00:00.000+0000 1.92 AAA 1998-03-10T00:00:00.000+0000 1.96 AAA 1998-03-11T00:00:00.000+0000 1.97 AAA 1998-03-12T00:00:00.000+0000 1.96 AAA 1998-03-13T00:00:00.000+0000 2.01 AAA 1998-03-16T00:00:00.000+0000 2.04 AAA 1998-03-17T00:00:00.000+0000 2.09 AAA 1998-03-18T00:00:00.000+0000 2.05 AAA 1998-03-19T00:00:00.000+0000 1.99 AAA 1998-03-20T00:00:00.000+0000 1.97 AAA 1998-03-23T00:00:00.000+0000 1.95 AAA 1998-03-24T00:00:00.000+0000 2.0 AAA 1998-03-25T00:00:00.000+0000 2.03 AAA 1998-03-26T00:00:00.000+0000 1.96 AAA 1998-03-27T00:00:00.000+0000 2.0 AAA 1998-03-30T00:00:00.000+0000 1.98 AAA 1998-03-31T00:00:00.000+0000 1.99 AAA 1998-04-01T00:00:00.000+0000 2.02 AAA 1998-04-02T00:00:00.000+0000 1.98 AAA 1998-04-03T00:00:00.000+0000 2.0 AAA 1998-04-06T00:00:00.000+0000 2.033 AAA 1998-04-07T00:00:00.000+0000 2.16 AAA 1998-04-08T00:00:00.000+0000 2.11 AAA 1998-04-09T00:00:00.000+0000 2.1 AAA 1998-04-14T00:00:00.000+0000 2.139 AAA 1998-04-15T00:00:00.000+0000 2.17 AAA 1998-04-16T00:00:00.000+0000 2.238 AAA 1998-04-17T00:00:00.000+0000 2.19 AAA 1998-04-20T00:00:00.000+0000 2.23 AAA 1998-04-21T00:00:00.000+0000 2.299 AAA 1998-04-22T00:00:00.000+0000 2.36 AAA 1998-04-23T00:00:00.000+0000 2.395 AAA 1998-04-24T00:00:00.000+0000 2.4 AAA 1998-04-27T00:00:00.000+0000 2.37 AAA 1998-04-28T00:00:00.000+0000 2.28 AAA 1998-04-29T00:00:00.000+0000 2.32 AAA 1998-04-30T00:00:00.000+0000 2.35 AAA 1998-05-01T00:00:00.000+0000 2.35 AAA 1998-05-04T00:00:00.000+0000 2.28 AAA 1998-05-05T00:00:00.000+0000 2.33 AAA 1998-05-06T00:00:00.000+0000 2.3 AAA 1998-05-07T00:00:00.000+0000 2.19 AAA 1998-05-08T00:00:00.000+0000 2.15 AAA 1998-05-11T00:00:00.000+0000 2.26 AAA 1998-05-12T00:00:00.000+0000 2.27 AAA 1998-05-13T00:00:00.000+0000 2.3 AAA 1998-05-14T00:00:00.000+0000 2.33 AAA 1998-05-15T00:00:00.000+0000 2.33 AAA 1998-05-18T00:00:00.000+0000 2.33 AAA 1998-05-19T00:00:00.000+0000 2.29 AAA 1998-05-20T00:00:00.000+0000 2.22 AAA 1998-05-21T00:00:00.000+0000 2.22 AAA 1998-05-22T00:00:00.000+0000 2.25 AAA 1998-05-25T00:00:00.000+0000 2.23 AAA 1998-05-26T00:00:00.000+0000 2.22 AAA 1998-05-27T00:00:00.000+0000 2.05 AAA 1998-05-28T00:00:00.000+0000 2.01 AAA 1998-05-29T00:00:00.000+0000 1.93 AAA 1998-06-01T00:00:00.000+0000 1.955 AAA 1998-06-02T00:00:00.000+0000 1

In [21]:
%sql drop table if exists timeseries;

In [22]:
focus_df.write.saveAsTable("timeseries")

In [23]:
from pyspark.sql.functions import year

df_2016 = ((df 
  .filter(year("Date") == '2016')
))

In [24]:
%sql drop table if exists asxhistoric2016;

In [25]:
df_2016.write.saveAsTable("asxhistoric2016")

In [26]:
%sql
SELECT COUNT(*) FROM asxhistoric2016;

count(1) 372892